In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [128]:
# parser v1
# parse records to list of dicts
# values mostly are strings
# easy to iterate through records
records = []
with open('Webscope_R6B/ydata-fp-td-clicks-v2_0.20111002') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        record = {}
        record['timestamp'] = components[0]
        record['displayed'] = components[1]
        record['clicked'] = int(components[2])
        # not using user feature
        record['user_features'] = []
        for i in range(4, len(components)):
            if components[i].startswith('|'):
                n = i
                break
            #record['user_features'].append(int(components[i]))
        record['pool'] = components[n:]    
        records.append(record)

In [152]:
len(records)

1607525

In [89]:
# parser that watches article lifespans
lives = {}
with open('Webscope_R6B/ydata-fp-td-clicks-v2_0.20111002') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        for component in components:
            if component.startswith('|id-'):
                timestamp = int(components[0])
                if component in lives.keys():
                    lives[component][1] = timestamp
                else:
                    lives[component] = [timestamp, timestamp]

In [22]:
lives.keys()

dict_keys(['|id-552077', '|id-555224', '|id-555528', '|id-559744', '|id-559855', '|id-560290', '|id-560518', '|id-560620', '|id-563115', '|id-563582', '|id-563643', '|id-563787', '|id-563846', '|id-563938', '|id-564335', '|id-564418', '|id-564604', '|id-565364', '|id-565479', '|id-565515', '|id-565533', '|id-565561', '|id-565589', '|id-565648', '|id-565747', '|id-565822', '|id-565930', '|id-566013', '|id-566022', '|id-566092', '|id-560805', '|id-564371', '|id-562265', '|id-565980', '|id-566431', '|id-566439', '|id-559833', '|id-566541', '|id-562374', '|id-566587', '|id-566478', '|id-566573', '|id-566602', '|id-562637', '|id-566631', '|id-566689', '|id-566726', '|id-566825', '|id-566838'])

In [125]:
for v in lives.values():
    print(v)

[1317513291, 1317515399]
[1317513291, 1317588299]
[1317513291, 1317587099]
[1317513291, 1317515699]
[1317513291, 1317587099]
[1317513291, 1317587099]
[1317513291, 1317578399]
[1317513291, 1317521099]
[1317513291, 1317581999]
[1317513291, 1317583799]
[1317513291, 1317599694]
[1317513291, 1317515699]
[1317513291, 1317599694]
[1317513291, 1317519299]
[1317513291, 1317571199]
[1317513291, 1317599694]
[1317513291, 1317520799]
[1317513291, 1317535499]
[1317513291, 1317583499]
[1317513291, 1317587399]
[1317513291, 1317583499]
[1317513291, 1317549899]
[1317513291, 1317577499]
[1317513291, 1317586799]
[1317513291, 1317570299]
[1317513291, 1317599694]
[1317517500, 1317582299]
[1317525000, 1317585899]
[1317527100, 1317594299]
[1317529500, 1317564299]
[1317565500, 1317599694]
[1317568200, 1317599694]
[1317569100, 1317599694]
[1317570600, 1317574799]
[1317572400, 1317599694]
[1317573300, 1317599694]
[1317577200, 1317599694]
[1317577800, 1317599694]
[1317578100, 1317599694]
[1317579300, 1317599694]


In [155]:
timestamp = 1317513600
life_rank = sorted(filter(lambda x: x[1][0]<timestamp, lives.items()), key= lambda x: x[1][1], reverse= True)
life_rank

[('|id-563643', [1317513291, 1317599694]),
 ('|id-563846', [1317513291, 1317599694]),
 ('|id-564418', [1317513291, 1317599694]),
 ('|id-565822', [1317513291, 1317599694]),
 ('|id-555224', [1317513291, 1317588299]),
 ('|id-565515', [1317513291, 1317587399]),
 ('|id-555528', [1317513291, 1317587099]),
 ('|id-559855', [1317513291, 1317587099]),
 ('|id-560290', [1317513291, 1317587099]),
 ('|id-565648', [1317513291, 1317586799]),
 ('|id-563582', [1317513291, 1317583799]),
 ('|id-565479', [1317513291, 1317583499]),
 ('|id-565533', [1317513291, 1317583499]),
 ('|id-563115', [1317513291, 1317581999]),
 ('|id-560518', [1317513291, 1317578399]),
 ('|id-565589', [1317513291, 1317577499]),
 ('|id-564335', [1317513291, 1317571199]),
 ('|id-565747', [1317513291, 1317570299]),
 ('|id-565561', [1317513291, 1317549899]),
 ('|id-565364', [1317513291, 1317535499]),
 ('|id-560620', [1317513291, 1317521099]),
 ('|id-564604', [1317513291, 1317520799]),
 ('|id-563938', [1317513291, 1317519299]),
 ('|id-5597

In [168]:
life_rank[:int(len(life_rank)*.3)]

[('|id-563643', [1317513291, 1317599694]),
 ('|id-563846', [1317513291, 1317599694]),
 ('|id-564418', [1317513291, 1317599694]),
 ('|id-565822', [1317513291, 1317599694]),
 ('|id-555224', [1317513291, 1317588299]),
 ('|id-565515', [1317513291, 1317587399]),
 ('|id-555528', [1317513291, 1317587099])]

In [172]:
# offline simulation for AG_L
def AG_L(n=1000000, m=25, life_threshold=.3):
    # records should be long enough
    '''
    n: number of turns to play
    m: number of turns to initialize
    '''
    global records
    global lives # dict of articles to their lifespans
    rewards = 0
    i = 0
    X = {} # expectations
    t = 1
    
    # initialization
    while t<=m:
        displayed = records[i]['displayed']
        clicked = records[i]['clicked']
        if displayed not in X.keys():
            X[displayed] = np.array([clicked, 1])
            print('initialized',displayed,':',X[displayed],'t/i:',t,i)
            t += 1
        i += 1
    
    while t<=n:
        best = max(X.items(), key= lambda x:x[1][0]/x[1][1])
        X_max = best[1][0]/best[1][1]
        print(t, X_max)
        if np.random.binomial(1, X_max): # exploit
            while i < len(records): # look for target arm
                displayed = records[i]['displayed']
                clicked = records[i]['clicked']
                if displayed == best[0]:
                    rewards += clicked
                    print('exploited', displayed, ':', clicked, 't/i:', t,i)
                    break
                i += 1
                
        else: # explore
            while i < len(records):
                displayed = records[i]['displayed']
                clicked = records[i]['clicked']
                time = int(records[i]['timestamp'])
                born = filter(lambda x: x[1][0]<time, lives.items()) # fileter out unborn articles from lives
                life_rank = sorted(born, key= lambda x: x[1][1], reverse= True) # sort born articles by their expiring date
                young = life_rank[ : int(len(life_rank)*life_threshold)] # articles among top $life_threshold$% on the life ranking
                if '|'+displayed in list(zip(*young))[0]:
                    rewards += clicked
                    print('explored', displayed, ':', clicked, 't/i:', t,i)
                    break
                i += 1

        X[displayed] = X.get(displayed, np.array([0, 0])) + np.array([clicked, 1])
        t += 1
        i += 1   
    return rewards

In [176]:
AG_L(n=1000)

initialized id-560620 : [0 1] t/i: 1 0
initialized id-565648 : [0 1] t/i: 2 1
initialized id-563115 : [0 1] t/i: 3 2
initialized id-552077 : [0 1] t/i: 4 3
initialized id-564335 : [0 1] t/i: 5 4
initialized id-565589 : [0 1] t/i: 6 5
initialized id-563938 : [1 1] t/i: 7 7
initialized id-560290 : [0 1] t/i: 8 8
initialized id-563643 : [0 1] t/i: 9 9
initialized id-565822 : [0 1] t/i: 10 15
initialized id-563787 : [0 1] t/i: 11 16
initialized id-555528 : [0 1] t/i: 12 21
initialized id-565364 : [0 1] t/i: 13 26
initialized id-559855 : [0 1] t/i: 14 30
initialized id-560518 : [0 1] t/i: 15 31
initialized id-565561 : [0 1] t/i: 16 32
initialized id-564604 : [0 1] t/i: 17 33
initialized id-563846 : [0 1] t/i: 18 54
initialized id-563582 : [0 1] t/i: 19 55
initialized id-565533 : [0 1] t/i: 20 57
initialized id-565479 : [0 1] t/i: 21 60
initialized id-559744 : [0 1] t/i: 22 65
initialized id-565515 : [0 1] t/i: 23 67
initialized id-564418 : [0 1] t/i: 24 69
initialized id-565747 : [0 1] t/i:

explored id-565822 : 0 t/i: 187 1390
188 0.10344827586206896
explored id-565515 : 0 t/i: 188 1391
189 0.10344827586206896
explored id-563643 : 0 t/i: 189 1400
190 0.1
explored id-555528 : 0 t/i: 190 1406
191 0.1
explored id-555528 : 0 t/i: 191 1407
192 0.1
explored id-555224 : 0 t/i: 192 1409
193 0.1
explored id-555224 : 0 t/i: 193 1411
194 0.1
explored id-565822 : 0 t/i: 194 1419
195 0.1
explored id-563846 : 0 t/i: 195 1425
196 0.1
explored id-565515 : 0 t/i: 196 1430
197 0.1
explored id-563846 : 0 t/i: 197 1433
198 0.1
explored id-563643 : 0 t/i: 198 1442
199 0.0967741935483871
explored id-565515 : 0 t/i: 199 1443
200 0.0967741935483871
explored id-563643 : 0 t/i: 200 1445
201 0.09375
explored id-563643 : 0 t/i: 201 1452
202 0.09090909090909091
explored id-565822 : 0 t/i: 202 1461
203 0.09090909090909091
explored id-555224 : 0 t/i: 203 1486
204 0.09090909090909091
explored id-564418 : 0 t/i: 204 1492
205 0.09090909090909091
explored id-555528 : 0 t/i: 205 1499
206 0.09090909090909091

explored id-565822 : 0 t/i: 445 3041
446 0.10294117647058823
explored id-555224 : 0 t/i: 446 3044
447 0.10294117647058823
explored id-563643 : 0 t/i: 447 3046
448 0.10144927536231885
exploited id-563643 : 0 t/i: 448 3058
449 0.1
explored id-563643 : 0 t/i: 449 3059
450 0.09859154929577464
explored id-565515 : 1 t/i: 450 3068
451 0.09859154929577464
explored id-555528 : 0 t/i: 451 3069
452 0.09859154929577464
explored id-563643 : 0 t/i: 452 3072
453 0.09722222222222222
explored id-555224 : 0 t/i: 453 3081
454 0.09722222222222222
explored id-563643 : 0 t/i: 454 3083
455 0.0958904109589041
explored id-555224 : 0 t/i: 455 3085
456 0.0958904109589041
explored id-565515 : 0 t/i: 456 3088
457 0.0958904109589041
explored id-555528 : 0 t/i: 457 3089
458 0.0958904109589041
explored id-563846 : 0 t/i: 458 3094
459 0.0958904109589041
explored id-565515 : 0 t/i: 459 3096
460 0.0958904109589041
explored id-565822 : 0 t/i: 460 3102
461 0.0958904109589041
explored id-555224 : 0 t/i: 461 3108
462 0.095

explored id-563643 : 0 t/i: 682 4356
683 0.09649122807017543
explored id-564418 : 0 t/i: 683 4358
684 0.09649122807017543
explored id-555528 : 0 t/i: 684 4359
685 0.09649122807017543
explored id-563846 : 0 t/i: 685 4363
686 0.09649122807017543
explored id-563643 : 0 t/i: 686 4365
687 0.09649122807017543
explored id-565822 : 0 t/i: 687 4366
688 0.09649122807017543
explored id-563643 : 0 t/i: 688 4367
689 0.09649122807017543
explored id-555528 : 0 t/i: 689 4368
690 0.09649122807017543
explored id-563643 : 0 t/i: 690 4369
691 0.09649122807017543
explored id-563846 : 0 t/i: 691 4371
692 0.09649122807017543
explored id-563846 : 0 t/i: 692 4372
693 0.09649122807017543
explored id-555224 : 1 t/i: 693 4374
694 0.09649122807017543
explored id-555224 : 1 t/i: 694 4375
695 0.09649122807017543
explored id-565822 : 0 t/i: 695 4379
696 0.09649122807017543
exploited id-565515 : 0 t/i: 696 4391
697 0.09615384615384616
exploited id-555224 : 1 t/i: 697 4393
698 0.10476190476190476
explored id-564418 : 0

937 0.09420289855072464
explored id-555224 : 0 t/i: 937 5793
938 0.09352517985611511
explored id-565515 : 0 t/i: 938 5794
939 0.09352517985611511
explored id-555528 : 0 t/i: 939 5795
940 0.09352517985611511
explored id-565515 : 0 t/i: 940 5799
941 0.09352517985611511
explored id-564418 : 0 t/i: 941 5801
942 0.09352517985611511
explored id-555528 : 0 t/i: 942 5804
943 0.09352517985611511
explored id-564418 : 0 t/i: 943 5806
944 0.09352517985611511
explored id-564418 : 0 t/i: 944 5811
945 0.09352517985611511
explored id-563846 : 0 t/i: 945 5814
946 0.09352517985611511
explored id-555224 : 0 t/i: 946 5815
947 0.09285714285714286
explored id-565515 : 0 t/i: 947 5820
948 0.09285714285714286
explored id-565822 : 0 t/i: 948 5825
949 0.09285714285714286
explored id-565822 : 0 t/i: 949 5827
950 0.09285714285714286
explored id-555224 : 0 t/i: 950 5828
951 0.09219858156028368
explored id-563643 : 0 t/i: 951 5832
952 0.09219858156028368
explored id-563643 : 0 t/i: 952 5839
953 0.09219858156028368


63